In [5]:
import geopandas as gpd
import pandas as pd
import os 
import glob
import requests
import zipfile
donwload_url = "https://catastotn.tndigit.it/export_semestrale_VL_PUBB/IDR0020230701_TIPOCATSH_CCXXX.zip"
url_csv = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSPeLuWTTF1JhWOhhR_ZJmSLBJhMqcJ771xWUeNnuX2co7aV2k2UytMRWU3AZdgfP4gIsWZZHsmx3T7/pub?output=csv"

In [6]:
df = pd.read_csv(url_csv)
codici_catastali = df.codice_comune_catastale.unique()

In [7]:
gdflist = []
for codice in codici_catastali:
    codice = str(codice).zfill(3)
    url = donwload_url.replace("XXX",codice)
    response = requests.get(url)
    # Elenca i nomi dei file che vuoi estrarre
    files_to_extract = []
    suffix = "_vl_uniqueparcel_poly"
    suffix = "_vl_parcel_poly"
    files_to_extract.append(codice + suffix + ".prj")
    files_to_extract.append(codice + suffix + ".shp")
    files_to_extract.append(codice + suffix + ".shx")
    files_to_extract.append(codice + suffix + ".dbf")
    if response.status_code == 200:
        with open("file.zip", "wb") as file:
            file.write(response.content)
        with zipfile.ZipFile("file.zip", "r") as zip_ref:
            for file_name in zip_ref.namelist():
                if file_name in files_to_extract:
                    zip_ref.extract(file_name)
        gdf = gpd.read_file(codice + suffix + ".shp")
        crs = gdf.crs
        gdflist.append(gdf)
        os.remove("file.zip")
        for shp in files_to_extract:
            os.remove(shp)
parcels = gpd.GeoDataFrame(pd.concat(gdflist, ignore_index=True), crs=crs)

In [8]:
parcels['catasto'] = ""
parcels['comune'] = "NO"
parcels['prg1'] = ""
parcels['prg2'] = ""
parcels['prg3'] = ""
parcels['prg4'] = ""
parcels['prg5'] = ""
parcels['aggiornamento'] = ""

for idx, row in df.iterrows():
    codice_comune_catastale= row['codice_comune_catastale']
    codice_particella = row['codice_particella']  
    p = parcels[(parcels.PT_CODE == codice_particella) & (parcels.PT_CCAT == codice_comune_catastale)]  
    if p.shape[0] >0:
        parcels.at[p.index[0],"comune"] = row['comune_ammistrativo']
        parcels.at[p.index[0],"catasto"] = row['nome_comune_catastale']
        parcels.at[p.index[0],"prg1"] = row['stato_destinazione_prg_1']
        parcels.at[p.index[0],"prg2"] = row['stato_destinazione_prg_2']
        parcels.at[p.index[0],"prg3"] = row['stato_destinazione_prg_3']
        parcels.at[p.index[0],"prg4"] = row['stato_destinazione_prg_4']
        parcels.at[p.index[0],"prg5"] = row['stato_destinazione_prg_5']
        parcels.at[p.index[0],"prg6"] = row['stato_destinazione_prg_6']
        parcels.at[p.index[0],"aggiornamento"] = row['data_ultimo_aggiornamento_dati']

In [14]:
parcels

,PT_TYPE,PT_CODE,PT_CCAT,PT_FABB,PT_PUB,geometry,catasto,comune,prg1,prg2,prg3,prg4,prg5,aggiornamento,prg6
0,P,.231,215,N,YES,"POLYGON ((655996.387 5078546.916, 655996.787 5...",,NO,,,,,,,NaN
1,P,.111/2,215,N,YES,"POLYGON ((656026.067 5078537.586, 656026.047 5...",,NO,,,,,,,NaN
2,P,86,215,N,YES,"POLYGON ((655629.111 5078555.637, 655629.474 5...",,NO,,,,,,,NaN
3,P,92/5,215,N,YES,"POLYGON ((655684.183 5078529.539, 655678.103 5...",,NO,,,,,,,NaN
4,P,.13/3,215,N,YES,"POLYGON ((655856.000 5078557.767, 655856.090 5...",,NO,,,,,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23166,C,.211,255,S,YES,"POLYGON ((661323.201 5088972.662, 661326.873 5...",,NO,,,,,,,NaN
23167,C,.408,255,S,YES,"POLYGON ((661304.365 5088950.650, 661302.427 5...",,NO,,,,,,,NaN
23168,C,.408,255,S,YES,"POLYGON ((661304.365 5088950.650, 661305.561 5...",,NO,,,,,,,NaN
23169,C,.432,255,S,YES,"POLYGON ((660733.359 5088051.420, 660729.412 5...",,NO,,,,,,,NaN


In [9]:
usi_civici = parcels[parcels.comune != "NO"]

In [12]:
usi_civici=usi_civici.to_crs(epsg=4326)

In [13]:
usi_civici.to_file("usi_civici.geojson")